In [ ]:
import time
import numpy as np
import random
from PIL import Image
from bldcsag512_SD15 import *


# - device는 SAM과 BLDCSAG768의 device를 다르게 설정해주세요.
SAM_device = 'cuda:2'
BLDCSAG_device = 'cuda:3'

## original 이미지 받아서 마스크 바로 생성

In [ ]:
# Define file prefix (can be: heel1, heel2, bag1, bottle1, car1, cup1, perfume1, perfume2)
file_prefix = "heel1"
background = "white"

# Define base paths
base_path = "/home/ada6k4_01/inpainting"
ori_path = f"{base_path}/file_ori_obj"
mask_path = f"{base_path}/files_mask1"
canny_path = f"{base_path}/canny"

# Define target size
target_size = (512, 512)

# Read and resize original image
img = Image.open(f"{ori_path}/{file_prefix}_white.png")
img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
img_resized.save(f"{ori_path}/resize_512/{file_prefix}_white.png")

# Read and resize mask
mask = create_mask(
    image_path=f"{mask_path}/{file_prefix}_mask.png",
    output_path=f"{mask_path}/invert/{file_prefix}_mask1.png",
    threshold=0,
)

mask = Image.open(f"{mask_path}/invert/{file_prefix}_mask1.png")
mask_resized = mask.resize(target_size, Image.Resampling.LANCZOS)
mask_resized.save(f"{mask_path}/real_mask_resize/{file_prefix}_mask1.png")

# Convert canny image to PIL and resize
canny_image = create_canny_mask(
    image_path=f"{ori_path}/{file_prefix}_white.png",
    output_path=f"{canny_path}/{file_prefix}_canny1.png",
    low_threshold=50,
    high_threshold=250,
)
canny_pil = Image.fromarray(canny_image)
canny_resized = canny_pil.resize(target_size, Image.Resampling.LANCZOS)
canny_resized.save(f"{canny_path}/resize/{file_prefix}_canny1.png")

plt.imshow(img_resized)
plt.title("Image Resized")
plt.axis('off')
plt.show()

plt.imshow(mask_resized, cmap='gray')
plt.title("Mask Resized")
plt.axis('off')
plt.show()

plt.imshow(canny_resized, cmap='gray')
plt.title("Canny Resized")
plt.axis('off')
plt.show()


In [ ]:
## 3 ## : BLDCSAG1024로 output image 생성
# - Generation time : 75~85s(w/.SAG), 50~60s(w/o.SAG) / GPU Memory Usage : 8500~9500MiB
# - prompt와 negative_prompt는1 자유롭게 변경하면서 사용해주세요.
# - output_col과 output_row는 output image의 개수와 출력 형태를 결정합니다.
# - blending_start_percentage에 따라 blending 과정을 얼마나 빠르게 시작할지 결정됩니다. (Default : 0.1, Recommend : [0.1,0.4])
# - kernel_size는 object outline에서 artifact 발견시 크기를 늘려주세요. (Default : 1, Recommend : [1,3])
# - num_inference_steps는 model이 inference 과정에서 진행할 sampling step입니다. (Default : 100, Recommend : [70,150])
# - guidance_scale는 prompt와 negative_prompt를 사용한 CFG Guidance scale 입니다. (Default : 7.0, Recommend : [7.0,7.5])
# - sag_scale은 SAG Guidance scale입니다. (Default : 0.6, Recommend : [0.6,0.8], SAG 미사용 시 0.0으로 설정해주시면 됩니다.)
# - seed는 None일 경우 random한 image가 generation됩니다.


parameters = {
    "prompt": "A heel at a stylish rooftop bar in New York City at night, offering a breathtaking view of the city's iconic skyline.",
    "negative_prompt": "high heel, person, human hand, human hair, human face, human arm, human leg, human body, handle mug, duplicates, duplicate, blurry, bokeh, cropped, deformed, error, lowres, jpeg artifacts, low resolution, low quality, cartoonish, drawing, water painting, painting, illustration",
    "output_col": 1,
    "output_row": 1,
    "blending_start_percentage": 0.1,
    "kernel_size": 1,  # Default : 1
    "num_inference_steps": 50,  # Default : 50
    "guidance_scale": 7.0,  # Default : 7.0
    "sag_scale": 0.6,  # Default : 0.6
    # "seed": random.randint(1, 2147483647),  # random.randint(1, 2147483647)   # Default : 0
    "seed": 1891752616,
}

start_time = time.time()
# - 주석처리된 부분은 참고용 input으로, 실제 input이 아니므로 주석해제하시면 오류 발생합니다.
bld_controlnet = BLDCSAG512(
    prompt=parameters["prompt"],
    negative_prompt=parameters["negative_prompt"],
    init_image=f"{ori_path}/resize_512/{file_prefix}_{background}.png",
    mask=f"{mask_path}/invert/{file_prefix}_mask1.png",
    # model_path='stabilityai/stable-diffusion-2-1',
    # controlnet_model_path='thibaud/controlnet-sd21-canny-diffusers',  # ControlNet SD 2.1 Canny model path
    blending_start_percentage=parameters["blending_start_percentage"],
    device=BLDCSAG_device,
    batch_size=parameters["output_col"] * parameters["output_row"],
    # output_path='output.png'
)
results = bld_controlnet.edit_image(
    kernel_size=parameters["kernel_size"],
    num_inference_steps=parameters["num_inference_steps"],
    guidance_scale=parameters["guidance_scale"],
    sag_scale=parameters["sag_scale"],
    seed=parameters["seed"],
)
end_time = time.time()
elapsed_time = end_time - start_time
parameters["generation_time"] = elapsed_time

image_saver = ImageGridSaver(
    results,
    parameters["output_col"],
    parameters["output_row"],
    bld_controlnet.output_path,
)
image_saver.save_grid()

img_display = ImageGridDisplay(
    f"{ori_path}/resize_512/{file_prefix}_{background}.png", 
    f"{mask_path}/invert/{file_prefix}_mask1.png",
    f"{canny_path}/resize/{file_prefix}_canny1.png", 
    results[0])
img_display.display()

table = ParametersTable(parameters)
table.display_table()
print(f'random seed: {parameters["seed"]}')

# evaluator = ImageQualityEvaluator(output_path='output.png', mask_path='mask.png', original_path='image.png')
# evaluator.show_metrics_table()

# 메모리 확인

In [24]:
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"Current GPU memory allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
        print(f"Max GPU memory allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB")
        print(f"Current GPU memory reserved: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")
        print(f"Max GPU memory reserved: {torch.cuda.max_memory_reserved() / 1024**2:.2f} MB")

# 한 번에 여러 개

In [ ]:
import time
import numpy as np
import random
from PIL import Image
from bldcsag512_SD15 import *


# - device는 SAM과 BLDCSAG768의 device를 다르게 설정해주세요.
SAM_device = 'cuda:2'
BLDCSAG_device = 'cuda:3'

In [ ]:
# 반복할 배경색 리스트 정의
backgrounds = ["white", "blue", "green", "red"]

for background in backgrounds:
    # Define file prefix (can be: heel1, heel2, bag1, bottle1, car1, cup1, perfume1, perfume2)
    object_prefix = "perfume"
    file_prefix = f"{object_prefix}1"

    # Define base paths
    base_path = "/home/ada6k4_01/inpainting"
    ori_path = f"{base_path}/file_ori_obj"
    mask_path = f"{base_path}/files_mask1"
    canny_path = f"{base_path}/canny"

    # Define target size
    target_size = (512, 512)

    # Read and resize original image
    img = Image.open(f"{ori_path}/{file_prefix}_{background}.png")
    img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
    img_resized.save(f"{ori_path}/resize_512/{file_prefix}_{background}.png")

    # Read and resize mask
    mask = create_mask(
        image_path=f"{mask_path}/{file_prefix}_mask.png",
        output_path=f"{mask_path}/invert/{file_prefix}_mask1.png",
        use_otsu=True,
        inver_mask=False,
        normalize=False,
    )

    mask = Image.open(f"{mask_path}/invert/{file_prefix}_mask1.png")
    mask_resized = mask.resize(target_size, Image.Resampling.NEAREST)
    mask_resized.save(f"{mask_path}/real_mask_resize/{file_prefix}_mask1.png")

    # Convert canny image to PIL and resize
    canny_image = create_canny_mask(
        image_path=f"{ori_path}/{file_prefix}_{background}.png",
        output_path=f"{canny_path}/{file_prefix}_canny1.png",
        low_threshold=50,
        high_threshold=250,
    )
    canny_pil = Image.fromarray(canny_image)
    canny_resized = canny_pil.resize(target_size, Image.Resampling.LANCZOS)
    canny_resized.save(f"{canny_path}/resize/{file_prefix}_canny1.png")

    parameters = {
        "prompt": f"A {object_prefix} is displayed on a rustic cobblestone street, with soft evening lantern light creating a cozy, nostalgic ambiance.",
        "negative_prompt": "person, human hand, human hair, human face, human arm, human leg, human body, handle mug, duplicates, duplicate, blurry, bokeh, cropped, deformed, error, lowres, jpeg artifacts, low resolution, low quality, cartoonish, drawing, water painting, painting, illustration",
        "output_col": 1,
        "output_row": 1,
        "blending_start_percentage": 0.1,
        "kernel_size": 3,
        "num_inference_steps": 50,
        "guidance_scale": 7.0,
        "sag_scale": 0.0,
        "seed": 354572332,
    }

    start_time = time.time()

    bld_controlnet = BLDCSAG512(
        prompt=parameters["prompt"],
        negative_prompt=parameters["negative_prompt"],
        init_image=f"{ori_path}/resize_512/{file_prefix}_{background}.png",
        mask=f"{mask_path}/real_mask_resize/{file_prefix}_mask1.png",
        blending_start_percentage=parameters["blending_start_percentage"],
        device=BLDCSAG_device,
        batch_size=parameters["output_col"] * parameters["output_row"],
        output_path=f"output_{background}.png",  # 배경색별 output 파일명 지정
    )
    

    results = bld_controlnet.edit_image(
        kernel_size=parameters["kernel_size"],
        num_inference_steps=parameters["num_inference_steps"],
        guidance_scale=parameters["guidance_scale"],
        sag_scale=parameters["sag_scale"],
        seed=parameters["seed"],
    )

    end_time = time.time()
    elapsed_time = end_time - start_time
    parameters["generation_time"] = elapsed_time

    # Save results
    image_saver = ImageGridSaver(
        results,
        parameters["output_col"],
        parameters["output_row"],
        bld_controlnet.output_path,
    )
    image_saver.save_grid()

    # Display results
    img_display = ImageGridDisplay(
        f"{ori_path}/resize_512/{file_prefix}_{background}.png",
        f"{mask_path}/real_mask_resize/{file_prefix}_mask1.png",
        f"{canny_path}/resize/{file_prefix}_canny1.png",
        results[0],
    )
    img_display.display()

    # Display parameters
    table = ParametersTable(parameters)
    table.display_table()
    print(f"Background: {background}")
    print(f'Random seed: {parameters["seed"]}')
    print("=" * 50)
    


# 여러 object 여러 background

In [ ]:
import time
import numpy as np
import random
from PIL import Image
from bldcsag512_SD15 import *


# - device는 SAM과 BLDCSAG768의 device를 다르게 설정해주세요.
SAM_device = 'cuda:2'
BLDCSAG_device = 'cuda:3'

In [ ]:
# 반복할 배경색 리스트 정의
backgrounds = ["white", "blue", "green", "red"]
# Define objects list with their prefixes
objects = ["heel1", "heel2", "bag1", "bottle1", "car1", "cup1", "perfume1", "perfume2"]

for object_prefix in objects:
    for background in backgrounds:
        # Define file prefix
        file_prefix = object_prefix

        # Define base paths
        base_path = "/home/ada6k4_01/inpainting"
        ori_path = f"{base_path}/file_ori_obj"
        mask_path = f"{base_path}/files_mask1"
        canny_path = f"{base_path}/canny"

        # Define target size
        target_size = (512, 512)

        # Read and resize original image
        img = Image.open(f"{ori_path}/{file_prefix}_{background}.png")
        img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
        img_resized.save(f"{ori_path}/resize_512/{file_prefix}_{background}.png")

        # Read and resize mask
        mask = create_mask(
            image_path=f"{mask_path}/{file_prefix}_mask.png",
            output_path=f"{mask_path}/invert/{file_prefix}_mask1.png",
            use_otsu=True,
            inver_mask=False,
            normalize=False,
        )

        mask = Image.open(f"{mask_path}/invert/{file_prefix}_mask1.png")
        mask_resized = mask.resize(target_size, Image.Resampling.NEAREST)
        mask_resized.save(f"{mask_path}/real_mask_resize/{file_prefix}_mask1.png")

        # Convert canny image to PIL and resize
        canny_image = create_canny_mask(
            image_path=f"{ori_path}/{file_prefix}_{background}.png",
            output_path=f"{canny_path}/{file_prefix}_canny1.png",
            low_threshold=50,
            high_threshold=250,
        )
        canny_pil = Image.fromarray(canny_image)
        canny_resized = canny_pil.resize(target_size, Image.Resampling.LANCZOS)
        canny_resized.save(f"{canny_path}/resize/{file_prefix}_canny1.png")

        # Customize prompt based on object type
        object_type = object_prefix.rstrip('123')  # Remove numbers from object name
        parameters = {
            "prompt": f"A {object_type} is displayed on a rustic cobblestone street, with soft evening lantern light creating a cozy, nostalgic ambiance.",
            "negative_prompt": "person, human hand, human hair, human face, human arm, human leg, human body, handle mug, duplicates, duplicate, blurry, bokeh, cropped, deformed, error, lowres, jpeg artifacts, low resolution, low quality, cartoonish, drawing, water painting, painting, illustration",
            "output_col": 1,
            "output_row": 1,
            "blending_start_percentage": 0.1,
            "kernel_size": 7,
            "num_inference_steps": 50,
            "guidance_scale": 7.0,
            "sag_scale": 0.0,
            "seed": 354572332,
        }

        start_time = time.time()

        bld_controlnet = BLDCSAG512(
            prompt=parameters["prompt"],
            negative_prompt=parameters["negative_prompt"],
            init_image=f"{ori_path}/resize_512/{file_prefix}_{background}.png",
            mask=f"{mask_path}/real_mask_resize/{file_prefix}_mask1.png",  # 배경색별 output 파일명 지정
            blending_start_percentage=parameters["blending_start_percentage"],
            device=BLDCSAG_device,
            batch_size=parameters["output_col"] * parameters["output_row"],
            output_path=f"/home/ada6k4_01/inpainting/output/output_{object_prefix}_{background}.png",  # Include object prefix in output filename
        )

        results = bld_controlnet.edit_image(
            kernel_size=parameters["kernel_size"],
            num_inference_steps=parameters["num_inference_steps"],
            guidance_scale=parameters["guidance_scale"],
            sag_scale=parameters["sag_scale"],
            seed=parameters["seed"],
        )

        end_time = time.time()
        elapsed_time = end_time - start_time
        parameters["generation_time"] = elapsed_time

        # Save results
        image_saver = ImageGridSaver(
            results,
            parameters["output_col"],
            parameters["output_row"],
            bld_controlnet.output_path,
        )
        image_saver.save_grid()

        # Display results
        img_display = ImageGridDisplay(
            f"{ori_path}/resize_512/{file_prefix}_{background}.png",
            f"{mask_path}/real_mask_resize/{file_prefix}_mask1.png",
            f"{canny_path}/resize/{file_prefix}_canny1.png",
            results[0],
        )
        img_display.display()

        # Display parameters
        # table = ParametersTable(parameters)
        # table.display_table()
        # print(f"Object: {object_prefix}")
        # print(f"Background: {background}")
        # print(f'Random seed: {parameters["seed"]}')
        # print("=" * 50)


In [4]:
# ## 3 ## : BLDCSAG1024로 output image 생성
# # - Generation time : 75~85s(w/.SAG), 50~60s(w/o.SAG) / GPU Memory Usage : 8500~9500MiB
# # - prompt와 negative_prompt는 자유롭게 변경하면서 사용해주세요.
# # - output_col과 output_row는 output image의 개수와 출력 형태를 결정합니다.
# # - blending_start_percentage에 따라 blending 과정을 얼마나 빠르게 시작할지 결정됩니다. (Default : 0.1, Recommend : [0.1,0.4])
# # - kernel_size는 object outline에서 artifact 발견시 크기를 늘려주세요. (Default : 1, Recommend : [1,3])
# # - num_inference_steps는 model이 inference 과정에서 진행할 sampling step입니다. (Default : 100, Recommend : [70,150])
# # - guidance_scale는 prompt와 negative_prompt를 사용한 CFG Guidance scale 입니다. (Default : 7.0, Recommend : [7.0,7.5])
# # - sag_scale은 SAG Guidance scale입니다. (Default : 0.6, Recommend : [0.6,0.8], SAG 미사용 시 0.0으로 설정해주시면 됩니다.)
# # - seed는 None일 경우 random한 image가 generation됩니다.


# parameters = {
#     "prompt": "A heel at a stylish rooftop bar in New York City at night, offering a breathtaking view of the city's iconic skyline.",
#     "negative_prompt": "person, human hand, human hair, human face, human arm, human leg, human body, handle mug, duplicates, duplicate, blurry, bokeh, cropped, deformed, error, lowres, jpeg artifacts, low resolution, low quality, cartoonish, drawing, water painting, painting, illustration",
#     "output_col": 1,
#     "output_row": 1,
#     "blending_start_percentage": 0.1,
#     "kernel_size": 1,  # Default : 1
#     "num_inference_steps": 50,  # Default : 50
#     "guidance_scale": 7.0,  # Default : 7.0
#     "sag_scale": 0.6,  # Default : 0.6
#     "seed": random.randint(1, 2147483647),  # random.randint(1, 2147483647)   # Default : 0
#     # "seed": 1446565216,
# }

# start_time = time.time()
# # - 주석처리된 부분은 참고용 input으로, 실제 input이 아니므로 주석해제하시면 오류 발생합니다.
# bld_controlnet = BLDCSAGXL(
#     prompt=parameters["prompt"],
#     negative_prompt=parameters["negative_prompt"],
#     # init_image='image.png',
#     # mask='mask.png',
#     # model_path='stabilityai/stable-diffusion-2-1',
#     # controlnet_model_path='thibaud/controlnet-sd21-canny-diffusers',  # ControlNet SD 2.1 Canny model path
#     blending_start_percentage=parameters["blending_start_percentage"],
#     device=BLDCSAG_device,
#     batch_size=parameters["output_col"] * parameters["output_row"],
#     # output_path='output.png'
# )
# results = bld_controlnet.edit_image(
#     kernel_size=parameters["kernel_size"],
#     num_inference_steps=parameters["num_inference_steps"],
#     guidance_scale=parameters["guidance_scale"],
#     # sag_scale=parameters["sag_scale"],
#     seed=parameters["seed"],
# )
# end_time = time.time()
# elapsed_time = end_time - start_time
# parameters["generation_time"] = elapsed_time

# image_saver = ImageGridSaver(
#     results,
#     parameters["output_col"],
#     parameters["output_row"],
#     bld_controlnet.output_path,
# )
# image_saver.save_grid()

# img_display = ImageGridDisplay("image1.png", "mask1.png", "canny.png", results[0])
# img_display.display()

# table = ParametersTable(parameters)
# table.display_table()
# print(f'random seed: {parameters["seed"]}')

# # evaluator = ImageQualityEvaluator(output_path='output.png', mask_path='mask.png', original_path='image.png')
# # evaluator.show_metrics_table()

## kernel 여러개 해보기

In [2]:
import time
import numpy as np
import random
from PIL import Image
from bldcsag512_SD15 import *


# - device는 SAM과 BLDCSAG768의 device를 다르게 설정해주세요.
SAM_device = 'cuda:2'
BLDCSAG_device = 'cuda:3'

/home/ada6k4_01/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ada6k4_01/anaconda3/envs/venv/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/ada6k4_01/anaconda3/envs/venv/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/ada6k4_01/anaconda3/envs/venv/lib/python3.10/site-packages/segment_anything_hq/modeling/tiny_vit_sam.py:662: UserWarning: Overwriting tiny_vit_

In [3]:
# 반복할 배경색 리스트 정의
backgrounds = ["white", "blue", "green", "red"]
# Define objects list with their prefixes
objects = ["heel1", "heel2", "bag1", "bottle1", "car1", "cup1", "perfume1", "perfume2"]
kernel_sizes = [3, 5]

for kernel_size in kernel_sizes:
    for object_prefix in objects:
        for background in backgrounds:
            # Define file prefix
            file_prefix = object_prefix

            # Define base paths
            base_path = "/home/ada6k4_01/inpainting"
            ori_path = f"{base_path}/file_ori_obj"
            mask_path = f"{base_path}/files_mask1"
            canny_path = f"{base_path}/canny"

            # Define target size
            target_size = (512, 512)

            # Read and resize original image
            img = Image.open(f"{ori_path}/{file_prefix}_{background}.png")
            img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
            img_resized.save(f"{ori_path}/resize_512/{file_prefix}_{background}.png")

            # Read and resize mask
            mask = create_mask(
                image_path=f"{mask_path}/{file_prefix}_mask.png",
                output_path=f"{mask_path}/invert/{file_prefix}_mask1.png",
                use_otsu=True,
                inver_mask=False,
                normalize=False,
            )

            mask = Image.open(f"{mask_path}/invert/{file_prefix}_mask1.png")
            mask_resized = mask.resize(target_size, Image.Resampling.NEAREST)
            mask_resized.save(f"{mask_path}/real_mask_resize/{file_prefix}_mask1.png")

            # Convert canny image to PIL and resize
            canny_image = create_canny_mask(
                image_path=f"{ori_path}/{file_prefix}_{background}.png",
                output_path=f"{canny_path}/{file_prefix}_canny1.png",
                low_threshold=50,
                high_threshold=250,
            )
            canny_pil = Image.fromarray(canny_image)
            canny_resized = canny_pil.resize(target_size, Image.Resampling.LANCZOS)
            canny_resized.save(f"{canny_path}/resize/{file_prefix}_canny1.png")

            # Customize prompt based on object type
            object_type = object_prefix.rstrip('123')  # Remove numbers from object name
            parameters = {
                "prompt": f"A {object_type} is displayed on a rustic cobblestone street, with soft evening lantern light creating a cozy, nostalgic ambiance.",
                "negative_prompt": "person, human hand, human hair, human face, human arm, human leg, human body, handle mug, duplicates, duplicate, blurry, bokeh, cropped, deformed, error, lowres, jpeg artifacts, low resolution, low quality, cartoonish, drawing, water painting, painting, illustration",
                "output_col": 1,
                "output_row": 1,
                "blending_start_percentage": 0.1,
                "kernel_size": kernel_size,  # Using the current kernel size from the loop
                "num_inference_steps": 50,
                "guidance_scale": 7.0,
                "sag_scale": 0.0,
                "seed": 354572332,
            }

            start_time = time.time()

            bld_controlnet = BLDCSAG512(
                prompt=parameters["prompt"],
                negative_prompt=parameters["negative_prompt"],
                init_image=f"{ori_path}/resize_512/{file_prefix}_{background}.png",
                mask=f"{mask_path}/real_mask_resize/{file_prefix}_mask1.png",
                blending_start_percentage=parameters["blending_start_percentage"],
                device=BLDCSAG_device,
                batch_size=parameters["output_col"] * parameters["output_row"],
                output_path=f"/home/ada6k4_01/inpainting/output/output_{object_prefix}_{background}_{kernel_size}.png",  # Modified to include kernel size
            )

            results = bld_controlnet.edit_image(
                kernel_size=parameters["kernel_size"],
                num_inference_steps=parameters["num_inference_steps"],
                guidance_scale=parameters["guidance_scale"],
                sag_scale=parameters["sag_scale"],
                seed=parameters["seed"],
            )

            end_time = time.time()
            elapsed_time = end_time - start_time
            parameters["generation_time"] = elapsed_time

            # Save results
            image_saver = ImageGridSaver(
                results,
                parameters["output_col"],
                parameters["output_row"],
                bld_controlnet.output_path,
            )
            image_saver.save_grid()

            # Display results
            # img_display = ImageGridDisplay(
            #     f"{ori_path}/resize_512/{file_prefix}_{background}.png",
            #     f"{mask_path}/real_mask_resize/{file_prefix}_mask1.png",
            #     f"{canny_path}/resize/{file_prefix}_canny1.png",
            #     results[0],
            # )
            # img_display.display()


Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  4.67it/s]


이미지가 /home/ada6k4_01/inpainting/output/output_heel1_white_3.png에 저장되었습니다.


Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  6.73it/s]


이미지가 /home/ada6k4_01/inpainting/output/output_heel1_blue_3.png에 저장되었습니다.


Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  6.58it/s]


...

이미지가 /home/ada6k4_01/inpainting/output/output_perfume2_blue_5.png에 저장되었습니다.


Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  5.33it/s]


이미지가 /home/ada6k4_01/inpainting/output/output_perfume2_green_5.png에 저장되었습니다.


Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  6.65it/s]


이미지가 /home/ada6k4_01/inpainting/output/output_perfume2_red_5.png에 저장되었습니다.


# CLIPAway 해보기

In [5]:
# from step2 import *
# ## 1 ## Mask generation을 위한 coord 확인
# output_image_path = "/home/ada6k4_01/inpainting/output.png"
# image_grid = ImageGrid(output_image_path)
# image_grid.draw_grid()  

# # - coord는 (x1, y1, x2, y2)로 값 순서를 반드시 지켜주세요. (Default: (2,2,14,14))
# img_processor = SamHQImageProcessor_CLIPAway(
#     img_path=output_image_path, 
#     device = SAM_device, 
#     coord=coord_fixed, 
#     mask_prefix='mask2', 
#     image_prefix='image2',
#     mask_threshold=0.95,
#     )
# img_processor.run()

In [6]:
# # 확장된 영역에서의 마스크 생성

# mask1 = np.array(Image.open("mask1.png"))
# mask2 = np.array(Image.open("mask2.png"))
# print("Mask1 Size:", mask1.shape)
# print("Mask2 Size:", mask2.shape)
# plt.imshow(mask2, cmap='gray')
# plt.show()
# mask_expansion = get_expansion_area_mask("mask1.png", "mask2.png")
# plt.imshow(mask_expansion, cmap='gray')
# plt.title("Expanded Mask Area")
# plt.axis('off')
# plt.show()


In [7]:
# 확장 마스크 부분만 다시 CLIPAway 적용
